## Actions

This section computes all of the Actions that will be executed on
the pi's specified in the Workbench below.

In [ ]:
import whendo.core.actions.file_action as file_x
import whendo.core.actions.logic_action as logic_x
import whendo.core.actions.sys_action as sys_x
import whendo_gpio.action as gpio_x

def compute_actions():
    actions = {}
    actions["pinA_on"] = gpio_x.SetPin(pin=27, on=True)
    actions["pinA_off"] = gpio_x.SetPin(pin=27, on=False)
    actions["pinA_state"] = gpio_x.SetPin(pin=27, on=False)
    actions["pinB_on"] = gpio_x.SetPin(pin=25, on=True)
    actions["pinB_off"] = gpio_x.SetPin(pin=25, on=False)
    actions["pinB_state"] = gpio_x.SetPin(pin=27, on=False)
    actions["gpio_clear"] = gpio_x.CleanupPins()
    actions["file_heartbeat"] = file_x.FileHeartbeat(file="gpio_beat.txt")
    actions["system_info"] = sys_x.SysInfo()
    actions["pause1"] = sys_x.Pause()
    actions["pause2"] = sys_x.Pause(seconds=2)
    actions["start_pivot"] = logic_x.All(
        action_list=[
            actions["pinA_on"],
            actions["pinB_on"],
            actions["pause1"],
            actions["pinB_on"],
        ]
    )
    actions["pause_gpio_clear"] = logic_x.All(
        action_list=[actions["pause2"], actions["gpio_clear"]]
    )
    return actions

## Schedulers

This section computes the schedulers that will determine when Actions are
executed.

In [ ]:
import whendo.core.schedulers.cont_scheduler as sched_x

def compute_schedulers():
    schedulers = {}
    schedulers["timely"] = sched_x.Timely(interval=10)
    schedulers["immediately"] = sched_x.Immediately()
    return schedulers

## Programs

This section computes the Programs that the pi's will run for
fixed time periods.

In [ ]:
import whendo.core.program as prog_x

def compute_programs():
    programs = {}
    programs["pivot_program"] = (
        prog_x.Program()
        .prologue("start_pivot")
        .epilogue("pause_gpio_clear")
        .body_element("timely", "pinA_on")
    )
    return programs

## Workbench and Clients

This section computes the Workbench containing the pi Clients. The Clients
are responsible for communicating with the pivot pi's.

In [ ]:
from whendo.sdk.workbench import Workbench
from whendo.sdk.client import Client

def compute_workbench():
    workbench = Workbench()
    workbench.add_client("pi3", Client(host="192.168.0.46"))
    workbench.add_client("pi4", Client(host="192.168.0.45"))
    return workbench

def reset(client:Client):
    client.clear_deferred_actions()
    client.clear_expired_actions()
    client.clear_dispatcher()
    client.clear_jobs()

def add_actions(client:Client):
    [ client.add_action(*action) for action in compute_actions().items() ]

def add_schedulers(client:Client):
    [ client.add_scheduler(*scheduler) for scheduler in compute_schedulers().items() ]

def add_programs(client:Client):
    [ client.add_program(*program) for program in compute_programs().items() ]

def initialize(client_list:[Client]):
    for c in client_list:
        reset(c)
        add_actions(c)
        add_schedulers(c)
        add_programs(c)

## Initialize all pivot pi's

Add Actions, Schedulers and Programs to pivot pi's for future scheduling.

In [ ]:
workbench = compute_workbench()
initialize(workbench.get_clients())

## It's Widget Time!

In [ ]:
from datetime import time, timedelta
from whendo.core.util import DateTime2, Now
dt2 = DateTime2(dt1=Now.dt(), dt2=Now.dt() + timedelta(seconds=45))
pi3, pi4 = [workbench.get_client(x) for x in ["pi3", "pi4"]]

In [ ]:
[x.schedule_program("pivot_program", dt2) for x in [pi3, pi4]]